In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import quandl
import pickle
import datetime as datetime

In [39]:
def df_add_first_diff(df):
    ' Adds the first differenced columns to the dataframe'
    diff_df = df.diff()
    df['d_six_m'] = diff_df['six_m']
    df['d_one_y'] = diff_df['one_y']
    df['d_two_y'] = diff_df['two_y']
    df['d_three_y'] = diff_df['three_y']
    df['d_five_y'] = diff_df['five_y']
    df['d_seven_y'] = diff_df['seven_y']
    df['d_ten_y'] = diff_df['ten_y']
    return df

In [62]:
def build_zeros_and_forwards(X):
    # Start building zero rate curves (spot rates)
    X_zeros = X[['three_m', 'six_m', 'one_y']].copy()
    X_fwds = X[['six_m', 'one_y']].copy()
    X_zeros['two_y'] = bootstrap_2yr(X)
    X_zeros['three_y'] = bootstrap_3yr(X, X_zeros)
    X_zeros['five_y'] = bootstrap_5yr(X, X_zeros)
    X_zeros['seven_y'] = bootstrap_7yr(X, X_zeros)
    X_zeros['ten_y'] = bootstrap_10yr(X, X_zeros)

    #bond_prices_3M, bond_prices_6M, bond_prices_1YR
    z_pr_6m  = zero_coupon_bond_price(par = 100, ytm = X_zeros['six_m'], time= 0.5)
    z_pr_1y  = zero_coupon_bond_price(par = 100, ytm = X_zeros['one_y'], time= 1.0)
    z_pr_2y  = zero_coupon_bond_price(par = 100, ytm = X_zeros['two_y'], time= 2.0)
    z_pr_3y  = zero_coupon_bond_price(par = 100, ytm = X_zeros['three_y'],time= 3.0)
    z_pr_5y  = zero_coupon_bond_price(par = 100, ytm = X_zeros['five_y'],time= 5.0)
    z_pr_7y  = zero_coupon_bond_price(par = 100, ytm = X_zeros['seven_y'], time= 7.0)
    z_pr_10y  = zero_coupon_bond_price(par = 100, ytm = X_zeros['ten_y'], time= 10.0)

    # taking the forward rates from the zero prices
    fwd_6_12 = z_pr_6m / z_pr_1y
    fwd_1_2= z_pr_1y/z_pr_2y
    fwd_2_3 =z_pr_2y/z_pr_3y
    fwd_3_5 =z_pr_3y/z_pr_5y
    fwd_5_7 =z_pr_5y/z_pr_7y
    fwd_7_10=z_pr_7y/z_pr_10y

    # now we need to account for periods that are not one year and adjusting to annualized rates
    fwd_6_12 = fwd_6_12**2
    fwd_3_5 =np.sqrt(fwd_3_5)
    fwd_5_7 =np.sqrt(fwd_5_7)
    fwd_7_10=fwd_7_10**(1/3)

    # Changing these back into interet rates from total returns
    X_fwds['one_y'] = fwd_6_12 - 1
    X_fwds['two_y'] = fwd_1_2 -1
    X_fwds['three_y'] = fwd_2_3 -1
    X_fwds['five_y'] = fwd_3_5 -1
    X_fwds['seven_y'] = fwd_5_7 -1
    X_fwds['ten_y'] = fwd_7_10 -1

    return X_fwds, X_zeros


In [41]:
def zero_coupon_bond_price(par, ytm, time):
    ''' Takes the par price, ytm and time to maturity and returns the spot price of the bond'''
    return par / (1 + ytm/2) ** (time*2)

In [42]:
def bootstrap_2yr(X):
    spots = np.zeros(shape=(len(X), 1))
    par = 100

    for i in range(len(X)):
        rate = 0
        r6 = X['six_m'].iloc[i]
        r12 = X['one_y'].iloc[i]
        cpn = X['two_y'].iloc[i] * par/2
        while True:
            rate += 0.0001
            delta = 100 - (cpn/((1+(r6/2))**1)) \
                    - (cpn/((1+(r12/2))**2)) \
                    - (cpn/((1+(rate / 2))**3)) \
                    - (cpn + par)/((1 + (rate / 2))**4)

            if delta >= 0:
                break
        spots[i] = rate
    return spots

In [43]:
def bootstrap_3yr(X, X_zeros):
    spots = np.zeros(shape=(len(X), 1))
    par = 100

    for i in range(len(X)):
        rate = 0
        r6 = X['six_m'].iloc[i]
        r12 = X['one_y'].iloc[i]
        r18 = X_zeros['two_y'].iloc[i]
        r24 = X_zeros['two_y'].iloc[i]
        cpn = X['three_y'].iloc[i] * par/2
        while True:
            rate += 0.0001
            delta = 100 - (cpn/((1+(r6/2))**1)) \
                    - (cpn/((1+(r12/2))**2)) \
                    - (cpn/((1+(r18 / 2))**3)) \
                    - (cpn/((1+(r24 / 2))**4)) \
                    - (cpn/((1+(rate / 2))**5)) \
                    - (cpn + par)/((1 + (rate / 2))**6)
            if delta >= 0:
                break
        spots[i] = rate
    return spots

In [44]:
def bootstrap_5yr(X, X_zeros):
    spots = np.zeros(shape=(len(X), 1))
    par = 100

    for i in range(len(X)):
        rate = 0
        r6 = X['six_m'].iloc[i]
        r12 = X['one_y'].iloc[i]
        r18 = X_zeros['two_y'].iloc[i]
        r24 = X_zeros['two_y'].iloc[i]
        r30 = X_zeros['three_y'].iloc[i]
        r36 = X_zeros['three_y'].iloc[i]

        cpn = X['five_y'].iloc[i] * par/2
        while True:
            rate += 0.00001
            delta = 100 - (cpn/((1+(r6/2))**1)) \
                    - (cpn/((1+(r12/2))**2)) \
                    - (cpn/((1+(r18 / 2))**3)) \
                    - (cpn/((1+(r24 / 2))**4)) \
                    - (cpn/((1+(r30 / 2))**5)) \
                    - (cpn/((1+(r36 / 2))**6)) \
                    - (cpn/((1+(rate / 2))**7)) \
                    - (cpn/((1+(rate / 2))**8)) \
                    - (cpn/((1+(rate / 2))**9)) \
                    - (cpn + par)/((1 + (rate / 2))**10)
            if delta >= 0:
                break
        spots[i] = rate
    return spots

In [45]:
def bootstrap_7yr(X, X_zeros):
    spots = np.zeros(shape=(len(X), 1))
    par = 100

    for i in range(len(X)):
        rate = 0
        r6 = X['six_m'].iloc[i]
        r12 = X['one_y'].iloc[i]
        r18 = X_zeros['two_y'].iloc[i]
        r24 = X_zeros['two_y'].iloc[i]
        r30 = X_zeros['three_y'].iloc[i]
        r36 = X_zeros['three_y'].iloc[i]
        r42 = X_zeros['five_y'].iloc[i]
        r48 = X_zeros['five_y'].iloc[i]
        r54 = X_zeros['five_y'].iloc[i]
        r60 = X_zeros['five_y'].iloc[i]

        cpn = X['seven_y'].iloc[i] * par/2
        while True:
            rate += 0.00001
            delta = 100 - (cpn/((1+(r6/2))**1)) \
                    - (cpn/((1+(r12/2))**2)) \
                    - (cpn/((1+(r18 / 2))**3)) \
                    - (cpn/((1+(r24 / 2))**4)) \
                    - (cpn/((1+(r30 / 2))**5)) \
                    - (cpn/((1+(r36 / 2))**6)) \
                    - (cpn/((1+(r42 / 2))**7)) \
                    - (cpn/((1+(r48 / 2))**8)) \
                    - (cpn/((1+(r54 / 2))**9)) \
                    - (cpn/((1+(r60 / 2))**10)) \
                    - (cpn/((1+(rate / 2))**11)) \
                    - (cpn/((1+(rate / 2))**12)) \
                    - (cpn/((1+(rate / 2))**13)) \
                    - (cpn + par)/((1 + (rate / 2))**14)
            if delta >= 0:
                break
        spots[i] = rate
    return spots

In [46]:
def bootstrap_10yr(X, X_zeros):
    spots = np.zeros(shape=(len(X), 1))
    par = 100

    for i in range(len(X)):
        rate = 0
        r6 = X['six_m'].iloc[i]
        r12 = X['one_y'].iloc[i]
        r18 = X_zeros['two_y'].iloc[i]
        r24 = X_zeros['two_y'].iloc[i]
        r30 = X_zeros['three_y'].iloc[i]
        r36 = X_zeros['three_y'].iloc[i]
        r42 = X_zeros['five_y'].iloc[i]
        r48 = X_zeros['five_y'].iloc[i]
        r54 = X_zeros['five_y'].iloc[i]
        r60 = X_zeros['five_y'].iloc[i]
        r66 = X_zeros['seven_y'].iloc[i]
        r72 = X_zeros['seven_y'].iloc[i]
        r78 = X_zeros['seven_y'].iloc[i]
        r84 = X_zeros['seven_y'].iloc[i]

        cpn = X['ten_y'].iloc[i] * par/2
        while True:
            rate += 0.00001
            delta = 100 - (cpn/((1+(r6/2))**1)) \
                    - (cpn/((1+(r12/2))**2)) \
                    - (cpn/((1+(r18 / 2))**3)) \
                    - (cpn/((1+(r24 / 2))**4)) \
                    - (cpn/((1+(r30 / 2))**5)) \
                    - (cpn/((1+(r36 / 2))**6)) \
                    - (cpn/((1+(r42 / 2))**7)) \
                    - (cpn/((1+(r48 / 2))**8)) \
                    - (cpn/((1+(r54 / 2))**9)) \
                    - (cpn/((1+(r60 / 2))**10)) \
                    - (cpn/((1+(r66 / 2))**11)) \
                    - (cpn/((1+(r72 / 2))**12)) \
                    - (cpn/((1+(r78 / 2))**13)) \
                    - (cpn/((1+(r84 / 2))**14)) \
                    - (cpn/((1+(rate / 2))**15)) \
                    - (cpn/((1+(rate / 2))**16)) \
                    - (cpn/((1+(rate / 2))**17)) \
                    - (cpn/((1+(rate / 2))**18)) \
                    - (cpn/((1+(rate / 2))**19)) \
                    - (cpn + par)/((1 + (rate / 2))**20)
            if delta >= 0:
                break
        spots[i] = rate
    return spots

In [47]:
df_treas = quandl.get("USTREASURY/YIELD", authtoken="zBYbsY7fujcHokgXQdsY",
        start_date = "2006-01-01", end_date="2019-03-28")


In [49]:
df_treas.describe()

,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
count,3313.000000,111.000000,3310.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3287.000000
mean,1.089119,2.387117,1.143752,1.252345,1.342668,1.540643,1.759547,2.215077,2.589713,2.938802,3.466085,3.626368
std,1.631714,0.072267,1.650751,1.672018,1.619399,1.478271,1.371656,1.193828,1.065508,0.975903,0.948785,0.812916
min,0.000000,2.210000,0.000000,0.020000,0.080000,0.160000,0.280000,0.560000,0.910000,1.370000,1.690000,2.110000
25%,0.040000,2.350000,0.060000,0.120000,0.190000,0.470000,0.830000,1.400000,1.880000,2.190000,2.680000,2.960000
50%,0.150000,2.410000,0.180000,0.310000,0.490000,0.870000,1.270000,1.840000,2.280000,2.710000,3.190000,3.360000
75%,1.680000,2.440000,1.787500,1.950000,2.120000,2.340000,2.450000,2.730000,3.090000,3.590000,4.360000,4.410000
max,5.270000,2.480000,5.190000,5.330000,5.300000,5.290000,5.260000,5.230000,5.230000,5.260000,5.440000,5.350000


In [50]:
df_treas['3 MO'] = df_treas['3 MO'].fillna(0.0001)

In [52]:
X = df_treas.copy()
# The following series are incomplete over the sample period and are removed
X = X.drop(['1 MO', '2 MO', '20 YR', '30 YR'], axis=1)

# transform to interest rates
X = X/100

In [53]:
X.describe()

,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR
count,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000
mean,0.011427,0.012523,0.013427,0.015406,0.017595,0.022151,0.025897,0.029388
std,0.016504,0.016720,0.016194,0.014783,0.013717,0.011938,0.010655,0.009759
min,0.000000,0.000200,0.000800,0.001600,0.002800,0.005600,0.009100,0.013700
25%,0.000600,0.001200,0.001900,0.004700,0.008300,0.014000,0.018800,0.021900
50%,0.001800,0.003100,0.004900,0.008700,0.012700,0.018400,0.022800,0.027100
75%,0.017800,0.019500,0.021200,0.023400,0.024500,0.027300,0.030900,0.035900
max,0.051900,0.053300,0.053000,0.052900,0.052600,0.052300,0.052300,0.052600


In [54]:
X = X.rename(columns = {'3 MO': 'three_m',
                        '6 MO': 'six_m',
                        '1 YR': 'one_y',
                        '2 YR': 'two_y',
                        '3 YR': 'three_y',
                        '5 YR': 'five_y',
                        '7 YR': 'seven_y',
                        '10 YR': 'ten_y'})

In [55]:
X.describe()

,three_m,six_m,one_y,two_y,three_y,five_y,seven_y,ten_y
count,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000
mean,0.011427,0.012523,0.013427,0.015406,0.017595,0.022151,0.025897,0.029388
std,0.016504,0.016720,0.016194,0.014783,0.013717,0.011938,0.010655,0.009759
min,0.000000,0.000200,0.000800,0.001600,0.002800,0.005600,0.009100,0.013700
25%,0.000600,0.001200,0.001900,0.004700,0.008300,0.014000,0.018800,0.021900
50%,0.001800,0.003100,0.004900,0.008700,0.012700,0.018400,0.022800,0.027100
75%,0.017800,0.019500,0.021200,0.023400,0.024500,0.027300,0.030900,0.035900
max,0.051900,0.053300,0.053000,0.052900,0.052600,0.052300,0.052300,0.052600


In [63]:
X_fwds, X_zeros  = build_zeros_and_forwards(X)

In [65]:
X_fwds.describe()

,six_m,one_y,two_y,three_y,five_y,seven_y,ten_y
count,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000,3313.000000
mean,0.012523,0.014444,0.017678,0.022241,0.029422,0.036375,0.039071
std,0.016720,0.015907,0.013664,0.012248,0.010659,0.009715,0.010204
min,0.000200,0.000900,0.002502,0.004105,0.009730,0.017387,0.018246
25%,0.001200,0.002502,0.007921,0.013559,0.022186,0.028230,0.030026
50%,0.003100,0.006612,0.012949,0.019413,0.027740,0.033950,0.037701
75%,0.019500,0.022931,0.024954,0.028101,0.038071,0.045670,0.049184
max,0.053300,0.053600,0.053497,0.053087,0.053215,0.055391,0.061604


In [67]:
X = df_add_first_diff(X)
X_zeros = df_add_first_diff(X_zeros)
X_fwds = df_add_first_diff(X_fwds)


In [68]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3313 entries, 2006-01-03 to 2019-03-28
Data columns (total 15 columns):
three_m      3313 non-null float64
six_m        3313 non-null float64
one_y        3313 non-null float64
two_y        3313 non-null float64
three_y      3313 non-null float64
five_y       3313 non-null float64
seven_y      3313 non-null float64
ten_y        3313 non-null float64
d_six_m      3312 non-null float64
d_one_y      3312 non-null float64
d_two_y      3312 non-null float64
d_three_y    3312 non-null float64
d_five_y     3312 non-null float64
d_seven_y    3312 non-null float64
d_ten_y      3312 non-null float64
dtypes: float64(15)
memory usage: 574.1 KB


In [69]:
X_fwds.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3313 entries, 2006-01-03 to 2019-03-28
Data columns (total 14 columns):
six_m        3313 non-null float64
one_y        3313 non-null float64
two_y        3313 non-null float64
three_y      3313 non-null float64
five_y       3313 non-null float64
seven_y      3313 non-null float64
ten_y        3313 non-null float64
d_six_m      3312 non-null float64
d_one_y      3312 non-null float64
d_two_y      3312 non-null float64
d_three_y    3312 non-null float64
d_five_y     3312 non-null float64
d_seven_y    3312 non-null float64
d_ten_y      3312 non-null float64
dtypes: float64(14)
memory usage: 388.2 KB


In [70]:
X_zeros.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3313 entries, 2006-01-03 to 2019-03-28
Data columns (total 15 columns):
three_m      3313 non-null float64
six_m        3313 non-null float64
one_y        3313 non-null float64
two_y        3313 non-null float64
three_y      3313 non-null float64
five_y       3313 non-null float64
seven_y      3313 non-null float64
ten_y        3313 non-null float64
d_six_m      3312 non-null float64
d_one_y      3312 non-null float64
d_two_y      3312 non-null float64
d_three_y    3312 non-null float64
d_five_y     3312 non-null float64
d_seven_y    3312 non-null float64
d_ten_y      3312 non-null float64
dtypes: float64(15)
memory usage: 414.1 KB
